# CP-SAT: Multiplications

In [1]:
from ortools.sat.python import cp_model

## Multiplication Equality Constraint

The solver works with a multiplication equality constraint of 2 variables. 

In [ ]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(1, 100, "y")
result = model.NewIntVar(0, 100 * 100, "result")

# We want to compute 22*11
model.Add(x == 22)
model.Add(y == 11)
model.AddMultiplicationEquality(result, [x, y])

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(result)}")


Solution is: 242


But with more than 2 terms, the solver will fail. This is despite the docs saying us that it should work. 

In [ ]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(0, 100, "y")
z = model.NewIntVar(0, 100, "z")
result = model.NewIntVar(0, 100 * 100 * 100, "result")

# We want to compute 22*11*33
model.Add(x == 22)
model.Add(y == 11)
model.Add(z == 33)
model.AddMultiplicationEquality(result, [x, y, z])

# The solver fails
solver = cp_model.CpSolver()
status = solver.Solve(model)
print(f"Status = {solver.StatusName(status)}")


Status = MODEL_INVALID


## Using an intermediate variable to apply MultiplicationEqualityConstraint with more than three variables

A trick is to use an intermediate variable. 

In [ ]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(0, 100, "y")
intermediate = model.NewIntVar(0, 100 * 100, "x*y")
z = model.NewIntVar(0, 100, "z")
result = model.NewIntVar(0, 100 * 100 * 100, "result")

# Let's say we want to compute 22*11*33
# First we compute x*y and store the result in an intermediate variable
model.Add(x == 22)
model.Add(y == 11)
model.AddMultiplicationEquality(intermediate, [x, y])
# And then we multiply this intermediate variable and get the final result
model.Add(z == 33)
model.AddMultiplicationEquality(result, [intermediate, z])

# The solver is happy now
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solution is: {solver.Value(result)}")

Status = OPTIMAL
Solution is: 7986


We can generalize this idea, and implement this recursive function. 

In [ ]:
from typing import List


def add_multiplication_constraint(
    model: cp_model.CpModel, target: cp_model.IntVar, variables: List[cp_model.IntVar],
):
    if len(variables) <= 2:
        # If less than 2 variables, we can add a normal inequality constraint
        model.AddMultiplicationEquality(target, variables)
        return
    else:
        last_variable = variables.pop()
        before_last_variable = variables.pop()
        # Use their bounds to define domain of the intermediate variable
        # You may need additional logic here to account for variable domain bounds
        ub = max(
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[0],
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[0],
        )
        lb = min(
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[0],
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[0],
        )
        # Create an intermediate variable
        intermediate = model.NewIntVar(
            lb=lb, ub=ub, name=f"{before_last_variable.Name()}*{last_variable.Name()}"
        )
        model.AddMultiplicationEquality(
            intermediate, [before_last_variable, last_variable]
        )
        # Recursion
        add_multiplication_constraint(model, target, variables + [intermediate])


Let's test it with the product of 10 variables with random values. 

In [ ]:
from random import randint 

model = cp_model.CpModel()

n_variables = 10
many_variables = [model.NewIntVar(-20, 20, f"{x}_{i}") for i in range(n_variables)]
result = model.NewIntVar(- 20 ** n_variables, 20 ** n_variables, "result")

# Set the var to be equal to random int between 1 and 10
real_product = 1
for i, var in enumerate(many_variables):
    random_value = randint(-20, 20)
    if random_value == 0:
        random_value += 1
    real_product *= random_value
    print(f"Set {x}_{i}={random_value}")
    model.Add(var == random_value)

add_multiplication_constraint(model, result, many_variables)

# The solver is happy now
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solution found by the solver is: {solver.Value(result)}")
print(f"Real solution is: {real_product}")

Set x_0=-19
Set x_1=17
Set x_2=-12
Set x_3=11
Set x_4=-1
Set x_5=11
Set x_6=-13
Set x_7=-12
Set x_8=-10
Set x_9=4
Status = OPTIMAL
Solution found by the solver is: 2926535040
Real solution is: 2926535040


## Big int limitations

Integers have limits, and you can reach them when you multiply many big numbers together.
 
I found the limit to be $2^{63} \approx 9,2 \times 10^{18}$. 

In [ ]:
# No problem
big_int = model.NewIntVar(- 2**63 + 1, 2**63 -1, "big_int")

In [ ]:
# Big trouble !!!
big_int = model.NewIntVar(0, 2**63, "big_int")

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. ortools.util.python.sorted_interval_list.Domain(arg0: int, arg1: int)

Invoked with: 0, 9223372036854775808

## Variable bounds

The previous code works when variables have fixed, integer bounds. But they can also be variables themselves.

In [ ]:
# TODO : multiplication when variables bounds are variables